<a href="https://colab.research.google.com/github/fabiorezendegithub/imersaoiaaluragooglegemini/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
%pip -q install google-genai

In [39]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [40]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [41]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Alura não tem uma data fixa para a próxima Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no [LinkedIn](https://br.linkedin.com/school/aluraonline/), [Instagram](https://www.instagram.com/aluraonline/), [YouTube](https://www.youtube.com/aluraonline), [X](https://twitter.com/AluraOnline), pois eles costumam divulgar os lançamentos de cursos e imersões por lá.
*   **Verificar a página de Inteligência Artificial da Alura:** Acesse a página de Inteligência Artificial da Alura ([https://www.alura.com.br/inteligencia-artificial](https://www.alura.com.br/inteligencia-artificial)) e procure por novidades e anúncios.
*   **Assinar a newsletter da Alura:** Ao se inscrever na newsletter, você receberá informações sobre os novos cursos e eventos da Alura diretamente no seu e-mail.

Fazendo isso, você não perderá a próxima Imersão IA com Google Gemini!

In [42]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Com base nas informações encontradas, a Alura e o Google realizaram diversas edições da Imersão IA com Google Gemini. A edição mais recente ocorreu em maio de 2025.

Para saber sobre as próximas edições, recomendo verificar diretamente no site da Alura ou nos canais de comunicação da Alura.

In [43]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: youtube.com, alura.com.br, youtube.com, educacao.mg.gov.br



In [44]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [45]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [46]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [47]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [48]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [49]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [50]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )

    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [51]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [55]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Geopolítica global
Maravilha! Vamos então criar o post sobre novidades em Geopolítica global

--- 📝 Resultado do Agente 1 (Buscador) ---



> Aqui estão algumas buscas que farei para encontrar os lançamentos mais relevantes em geopolítica global:
> 
> 
> Com base nas notícias mais recentes, aqui estão 5 desenvolvimentos geopolíticos globais relevantes:
> 
> 1.  **A ascensão do Sul Global**: O Brasil tem se posicionado cada vez mais como parte do Sul Global, buscando parcerias com China e Rússia. A China tem aumentado seus investimentos no Brasil, principalmente nos setores de energia, infraestrutura e tecnologia. O Brasil sediará o Fórum Parlamentar do BRICS em junho. No entanto, essa postura pode acarretar hostilidade do Ocidente.
> 2.  **O retorno de Donald Trump**: A eleição de Donald Trump para a presidência dos EUA trouxe consigo uma onda de isolacionismo. Suas ações e decisões (ou omissões) terão um impacto significativo nos eventos de política internacional em 2025. Trump tem uma postura protecionista no comércio exterior, o que gerou tensões comerciais e disputas na OMC. Ele também questionou a aliança dos EUA com a OTAN, exigindo que os países membros aumentassem seus investimentos militares.
> 3.  **Rivalidade entre EUA e China**: A disputa tarifária entre EUA e China continua por hegemonia global. A China avança rapidamente em sua agenda de desenvolvimento tecnológico, enquanto os Estados Unidos enfrentam o dilema de manter a liderança tecnológica em um mundo cada vez mais descentralizado. A relação entre os dois países pode ser de rompimento.
> 4.  **Conflitos armados e instabilidade geopolítica**: O conflito armado entre estados é considerado o risco global mais urgente para 2025. Os conflitos na Ucrânia, no Oriente Médio e no Sudão mantêm uma pressão constante sobre a segurança do comércio. A instabilidade geopolítica e os conflitos afetam o comércio global, com o Mar Vermelho e o Canal de Suez se tornando pontos críticos de tensão.
> 5.  **Inteligência Artificial (IA)**: A IA está acelerando a diferença entre a América Latina e as potências globais. A IA avança, mas sem que os governos tenham criado salvaguardas ou regras adequadas. A deterioração na cooperação global colabora para isso.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos que você forneceu, vou criar um plano de conteúdo detalhado para um post de mídia social sobre o tema mais relevante.
> 
> Para garantir que eu tenha uma compreensão completa e possa gerar o melhor plano possível, usarei a ferramenta de busca do Google para aprofundar cada tópico.
> 
> 
> Após analisar os resultados da pesquisa, o tema mais relevante para um post de mídia social impactante sobre geopolítica global é:
> 
> **4. Conflitos armados e instabilidade geopolítica**: Este tema é identificado como o risco global mais urgente para 2025. Os conflitos na Ucrânia, no Oriente Médio e no Sudão exercem pressão constante sobre a segurança do comércio. A instabilidade geopolítica e os conflitos afetam o comércio global, com o Mar Vermelho e o Canal de Suez se tornando pontos críticos de tensão.
> 
> **Pontos mais relevantes:**
> 
> *   A escalada das tensões geopolíticas é um dos aspectos mais alarmantes, com conflitos armados entre estados classificados como o principal risco atual.
> *   O aumento dos conflitos armados na última década, juntamente com eventos climáticos extremos, polarização social e avanços tecnológicos que facilitam a desinformação, contribuem para um cenário global instável.
> *   A instabilidade geopolítica e os conflitos afetam o comércio global, com o Mar Vermelho e o Canal de Suez se tornando pontos críticos de tensão.
> *   Os conflitos armados não apenas afetam as cadeias de suprimentos globais, mas também criam um ambiente de incerteza que exige uma gestão de crise eficaz.
> 
> **Plano de Conteúdo para o Post:**
> 
> 1.  **Título:** "Crise Global: Conflitos Armados e a Instabilidade Geopolítica em 2025"
> 2.  **Introdução:**
>     *   Comece com uma frase impactante sobre como os conflitos armados são o risco global mais urgente em 2025.
>     *   Mencione os conflitos em andamento na Ucrânia, no Oriente Médio e no Sudão como exemplos.
> 3.  **Desenvolvimento:**
>     *   Explique como a escalada das tensões geopolíticas contribui para a instabilidade global.
>     *   Discuta como os conflitos armados afetam a segurança do comércio e as cadeias de suprimentos globais.
>     *   Destaque os pontos críticos de tensão, como o Mar Vermelho e o Canal de Suez.
>     *   Apresente dados e estatísticas do Fórum Econômico Mundial sobre os riscos globais em 2025.
> 4.  **Impacto:**
>     *   Analise como a instabilidade geopolítica afeta a economia global, a segurança alimentar e os deslocamentos populacionais.
>     *   Discuta a importância da gestão de crise e da resiliência para as empresas e organizações em um ambiente de incerteza.
> 5.  **Conclusão:**
>     *   Enfatize a necessidade de colaboração global para mitigar os riscos e construir um futuro mais seguro.
>     *   Incentive os seguidores a compartilhar suas opiniões e a se manterem informados sobre os desenvolvimentos geopolíticos.
> 6.  **Chamada para Ação:**
>     *   Faça perguntas para estimular o engajamento, como: "Como você acha que os conflitos globais afetarão nosso futuro?" ou "Quais medidas podemos tomar para promover a paz e a estabilidade?".
> 7.  **Hashtags:**
>     *   Use hashtags relevantes como #geopolítica, #conflitosarmados, #riscosglobais, #instabilidade, #segurançaglobal, #comérciointernacional, #fórumeconômicomundial, #gestãodecrise, #resiliência, #colaboraçãoglobal.
> 
> Este plano de conteúdo fornecerá um post informativo e envolvente que ajudará a aumentar a conscientização sobre os desafios geopolíticos globais e a importância da colaboração para enfrentá-los.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚨 **ALERTA GLOBAL: O mundo em conflito e o impacto na nossa vida!** 🚨
> 
> Você sabia que a escalada das tensões geopolíticas é o maior risco global de 2024? 🤯 Os conflitos na Ucrânia, no Oriente Médio e no Sudão não são apenas manchetes distantes – eles afetam diretamente o comércio, a economia e a nossa segurança!
> 
> ⚠️ **Pontos críticos:** O Mar Vermelho e o Canal de Suez, rotas cruciais para o comércio mundial, estão sob tensão constante. Isso significa cadeias de suprimentos mais lentas, produtos mais caros e um futuro incerto.
> 
> 📈 **O que podemos fazer?** Precisamos estar informados e preparados! A instabilidade global exige que empresas e governos ajam com rapidez e inteligência. A colaboração global é fundamental para construir um futuro mais seguro e estável.
> 
> 🗣️ **Sua vez!** Como você acha que os conflitos globais vão impactar o nosso futuro? Quais medidas podemos tomar para promover a paz e a estabilidade? Compartilhe sua opinião nos comentários! 👇
> 
> #geopolítica #conflitosarmados #riscosglobais #segurançaglobal
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda melhor para engajar o público jovem do Instagram. Aqui estão algumas sugestões:
> 
> *   **Tom:** O tom está um pouco alarmista. Podemos suavizar um pouco para não assustar tanto o público, mantendo a seriedade do tema.
> *   **Emojis:** Estão ok, mas podemos diversificar um pouco para quebrar a seriedade do tema.
> *   **Hashtags:** Estão boas, mas podemos adicionar algumas mais específicas para aumentar o alcance.
> *   **Call to action:** Podemos ser mais específicos sobre como o público pode se informar e fazer a diferença.
> 
> **Sugestão de Rascunho Revisado:**
> 
> 🌍 **GEOPOLÍTICA: Por que os conflitos no mundo importam pra gente?** 🌍
> 
> Já parou pra pensar como as tensões lá fora afetam o nosso dia a dia? 🤯 Em 2024, a escalada geopolítica é um dos maiores riscos globais, e os conflitos na Ucrânia, no Oriente Médio e no Sudão têm impacto direto no comércio, na economia e até na nossa segurança! 😥
> 
> ⚠️ **Fica ligado:** O Mar Vermelho e o Canal de Suez, que são super importantes pro comércio mundial, estão em áreas de tensão. Isso pode deixar as coisas mais caras e gerar um futuro incerto. 😕
> 
> 📈 **E aí, o que a gente pode fazer?** Se informar e se preparar! Empresas e governos precisam agir rápido e com inteligência pra lidar com essa instabilidade. A união faz a força pra construir um futuro mais seguro e estável. 💪
> 
> 🗣️ **Sua vez!** Como você acha que esses conflitos vão mexer com o nosso futuro? Quais atitudes podemos tomar pra ajudar a promover a paz? Deixe sua opinião nos comentários! 👇
> 
> \#geopolítica \#conflitos \#notíciasinternacionais \#paz \#futuro \#segurançaglobal \#economia \#comércioexterior
> 


--------------------------------------------------------------
